In [1]:
pip install pennylane --upgrade

     |████████████████████████████████| 631 kB 34.2 MB/s 
     |████████████████████████████████| 4.4 MB 37.3 MB/s 


In [2]:
#general import
import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

np.random.seed(42)

In [3]:
X, y = load_iris(return_X_y=True)

# pick inputs and labels from the first two classes only,
# corresponding to the first 100 samples
X = X[:100]
y = y[:100]

# For now a Standard Scaler always applied, if not will be specified
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
#This will change in some cases, KEEP AN EYE!
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [4]:
print('Shape X_train:', X_train.shape)
print('Shape Y_train', y_train.shape)
print('Shape X_test:', X_test.shape)
print('Shape Y_test:', y_test.shape)

Shape X_train: (75, 4)
Shape Y_train (75,)
Shape X_test: (25, 4)
Shape Y_test: (25,)


In [5]:
y_test

array([ 1.,  1.,  1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,  1.,
       -1.,  1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.])

# Model 2: Kernel with Amplitude Encoding and 2 Qubits

In [7]:
from pennylane.templates import AmplitudeEmbedding, BasisEmbedding, DisplacementEmbedding

n_qubits = 2
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    AmplitudeEmbedding(x1, wires=range(n_qubits), normalize=True)
    qml.adjoint(AmplitudeEmbedding)(x2, wires=range(n_qubits), normalize=True)
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [8]:
kernel(X_train[0], X_train[0])

tensor(1., requires_grad=True)

In [9]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [10]:
svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [11]:
predictions = svm.predict(X_test)
Acc = accuracy_score(predictions, y_test)
print('The accuracy is: ', Acc)

The accuracy is:  0.44


The Accuracy falls down a lot! We remember that we did reduce, from the experiment before, the number of qubits from 4 to 2.

In [13]:
dev_kernel.num_executions #here there is a worse model and even same evaluations!

7501